In [1]:
#Load all necessary libraries
import numpy as np
import cv2 # opencv
import os # control and access the directory structure in local machine
from matplotlib import pyplot as plt
import time


In [7]:
os.chdir('C:/Users/rithanya/Documents/Python/faces/Employee_dataset/16_2_2018/') #folder where I unzipped data.zip

#OpenCV cascades (xml files) are typically stored at this location
#C:\Users\<user>\Miniconda3\Library\etc\haarcascades
haarcascades_path = os.listdir('C:/Users/rithanya/Miniconda3/Library/etc/haarcascades/')
frontface_alt_cascade = 'C:/Users/rithanya/Miniconda3/Library/etc/haarcascades/haarcascade_frontalface_alt.xml'
frontface_default_cascade = 'C:/Users/rithanya/Miniconda3/Library/etc/haarcascades/haarcascade_frontalface_default.xml'
frontface_alt2_cascade = 'C:/Users/rithanya/Miniconda3/Library/etc/haarcascades/haarcascade_frontalface_alt2.xml'
frontface_alt_tree_cascade = 'C:/Users/rithanya/Miniconda3/Library/etc/haarcascades/haarcascade_frontalface_alt2.xml'

lbpcascadesPath = os.listdir("C:/Users/rithanya/Miniconda3/Library/etc/lbpcascades/")

lbp_frontface_path = "C:/Users/rithanya/Miniconda3/Library/etc/lbpcascades/lbpcascade_frontalface.xml"
lbp_fronface_improved_path = "C:/Users/rithanya/Miniconda3/Library/etc/lbpcascades/lbpcascade_frontalface_improved.xml"
lbp_frontface_profile_path = "C:/Users/rithanya/Miniconda3/Library/etc/lbpcascades/lbpcascade_profileface.xml"

In [3]:
#utility function to apply differenct cascade function on the images at difference scaleFactor
def detect(faceCascade, gray_,  scaleFactor_ = 1.1):
    faces = faceCascade.detectMultiScale(
                    gray_,
                    scaleFactor= scaleFactor_,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )
    return faces

In [4]:
# code that iterates thru the images in the celebrityfaces dataset and detects faces. Finally it
# ... only displays those images that it can't detect the faces.
lbp_frontfaceCascade = cv2.CascadeClassifier(lbp_frontface_path)
lbp_fronfaceimprovedCascade = cv2.CascadeClassifier(lbp_fronface_improved_path)
lbp_profileCascade = cv2.CascadeClassifier(lbp_frontface_profile_path)



In [5]:
#function to do data augumentation on images
def imgaug(img):
    #          Rotate process
    img_list = []


    #code to rotate the image
    ang_range=60
    rows,cols = img.shape
    ang_rot = np.random.uniform(ang_range)-ang_range/2
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)
    img_rotate = cv2.warpAffine(img,Rot_M,(cols,rows))
    img_list.append(img_rotate)

    #code to flip the image
    img_flip = cv2.flip(img, 1)
    img_list.append(img_flip)

    #code to do shear

    #code to do distortion

    return img_list


In [79]:
# Code to perform data augumentation for different employees
from os.path import basename
from matplotlib import pyplot as plt

img_folder_path = 'Employee_dataset/lalitha'
i = 0
for filename in os.listdir(img_folder_path):# iterate thru each image in the employee folder
    filename = img_folder_path + '/' + filename # build the path to the image file
    if(filename.lower().endswith('.jpg')):
       #reading and displaying the original
        print(filename)
        img = cv2.imread(filename,0) # read the image using OpenCV
        #plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
        #plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
        #plt.show()

        img_list = imgaug(img)
        
        for pic in img_list:
            i = i + 1
            save_path = img_folder_path + "/" + os.path.basename(filename).split('.')[0] +"_aug" + str(i) +".jpg"
            print(save_path)
            cv2.imwrite(save_path, pic)
            continue
            plt.imshow(pic, cmap = 'gray', interpolation = 'bicubic')
            plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
            plt.show()
    continue
#continue

Employee_dataset/lalitha/Snapshot_20180208_83.JPG
Employee_dataset/lalitha/Snapshot_20180208_83_aug1.jpg
Employee_dataset/lalitha/Snapshot_20180208_83_aug2.jpg
Employee_dataset/lalitha/Snapshot_20180208_84.JPG
Employee_dataset/lalitha/Snapshot_20180208_84_aug3.jpg
Employee_dataset/lalitha/Snapshot_20180208_84_aug4.jpg
Employee_dataset/lalitha/Snapshot_20180208_85.JPG
Employee_dataset/lalitha/Snapshot_20180208_85_aug5.jpg
Employee_dataset/lalitha/Snapshot_20180208_85_aug6.jpg
Employee_dataset/lalitha/Snapshot_20180208_86.JPG
Employee_dataset/lalitha/Snapshot_20180208_86_aug7.jpg
Employee_dataset/lalitha/Snapshot_20180208_86_aug8.jpg
Employee_dataset/lalitha/Snapshot_20180208_87.JPG
Employee_dataset/lalitha/Snapshot_20180208_87_aug9.jpg
Employee_dataset/lalitha/Snapshot_20180208_87_aug10.jpg
Employee_dataset/lalitha/Snapshot_20180208_88.JPG
Employee_dataset/lalitha/Snapshot_20180208_88_aug11.jpg
Employee_dataset/lalitha/Snapshot_20180208_88_aug12.jpg
Employee_dataset/lalitha/Snapshot_201

In [6]:
#function to detect face using OpenCV
def detect_face(img, face_cascade):
    #convert the test image to gray scale as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow: Haar classifier
    #face_cascade = cv2.CascadeClassifier('opencv-files/lbpcascade_frontalface.xml')

    #let's detect multiscale images(some images may be closer to camera than others)
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5);

    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None

    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]

    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [41]:
#this function will read all persons' training images, detect face from each image
#and will return two lists of exactly same size, one list 
#of faces and another list of labels for each face
def prepare_training_data(trainfolder = './'):
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    label_names = ['']
    total_images = 0
    #lbp_frontfaceCascade = cv2.CascadeClassifier(lbp_frontface_path)
    haar_faceCascade_default = cv2.CascadeClassifier(frontface_default_cascade)
    haar_frontface_alt = cv2.CascadeClassifier(frontface_alt_cascade)
    #detected_images = []
    for imgfolder in [name for name in os.listdir(trainfolder) if os.path.isdir(name)]: #iterate thru each of the 5 celeb folders
        if(imgfolder != '.DS_Store'):
            label_names.append(imgfolder)
            #print(imgfolder)
            #for filename in os.listdir(trainfolder + imgfolder + "/samples"):# iterate thru each image in a celeb folder
            #    filename = trainfolder + imgfolder + '/samples/' + filename # build the path to the image file
            for filename in os.listdir(trainfolder + imgfolder ):# iterate thru each image in a celeb folder
                filename = trainfolder + imgfolder + '/' + filename # build the path to the image file
                if(filename.lower().endswith('.jpg')):
                    print(filename.split('.')[0])
                    face = cv2.imread(filename) # read the image using OpenCV
                    #add face to list of faces
                    if(face is not None):
                        gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                        #continue
                        total_images = total_images + 1
                        faces.append(gray)
                        #add label for this face
                        labels.append(label_names.index(imgfolder))
                        
                    #print(label_names.index(imgfolder))
                    continue

                    plt.imshow(face, cmap = 'gray', interpolation = 'bicubic') # display all images read
                    plt.xticks([]), plt.yticks([])
                    plt.show()
    return faces, labels, label_names, total_images

In [42]:
#code to load the dataset and detect the faces and labels for training
print("Preparing data...")

faces, labels, label_names, total_images = prepare_training_data('./')
print("Data prepared")

#print total faces and labels
print("Detected faces: ", len(faces))
print("Detected labels: ", len(labels))
print("Total Images: ", total_images)
#print(labels)

Preparing data...
akash
aravind
Arun_S
Dinesh_Nagarajan
gowtham
harini
Jeffrey_Rajan
karthick_aravindan
karthic_chandran
Karthik
keerthana
lalitha
Mani
Ragav
Saravanan
Steve
Data prepared
Detected faces:  291
Detected labels:  291
Total Images:  291


In [60]:
#function to draw rectangle on image 
#according to given (x, y) coordinates and 
#given width and heigh
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
 
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
     cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [28]:
#Code to perform cross validation on the training set to get training score...
def cross_val(cv = 5):
    scores = []
    while(cv > 0):
        cv = cv - 1
        from sklearn.cross_validation import train_test_split
        Xtrain, Xtest, ytrain, ytest = train_test_split(faces, labels)
        print(len(Xtrain))
        print(len(Xtest))
        #print(ytrain)
        #print(ytest)
        face_recognizer = cv2.face.LBPHFaceRecognizer_create()
        face_recognizer.train(Xtrain, np.array(ytrain))
        test_labels = []
        pred_labels = []
        i = -1
        for face in Xtest:
            i = i + 1
            label = face_recognizer.predict(face)
            label_text = label_names[label[0]]
            #face, label = predict_train(face)
            #print(label_names)
            #print(ytest[i])
            test_labels.append(label_names[ytest[i]])
            pred_labels.append(label_text)

        from sklearn.metrics import accuracy_score
        scores.append(accuracy_score(test_labels, pred_labels))
    return scores

        #from sklearn.metrics import confusion_matrix
        #print(confusion_matrix(test_labels, pred_labels))


In [34]:
print('Cross Validation to get Training Score...')
scores =  cross_val(2)
print (scores)
print(np.mean(scores))


Cross Validation to get Training Score...
1110
371
1110
371
[0.9703504043126685, 0.95687331536388143]
0.963611859838


In [35]:
#code to persist the trained model to use it later
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))
face_recognizer.write('C:/Users/rithanya/Documents/Python/faces/HackObsV1.yml')
face_recognizer2 = cv2.face.LBPHFaceRecognizer_create()
face_recognizer2.read('C:/Users/rithanya/Documents/Python/faces/HackObsV1.yml')

test_labels = []
pred_labels = []
print(label_names)
i = -1
for face in faces:
    i = i + 1
    label = face_recognizer2.predict(face)
    pred_labels.append(label[0])

from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred_labels))

['', 'akash', 'aravind', 'Arun_S', 'Dinesh_Nagarajan', 'gowtham', 'harini', 'Jeffrey_Rajan', 'karthick_aravindan', 'karthic_chandran', 'Karthik', 'keerthana', 'lalitha', 'Mani', 'Saravanan', 'Steve']
1.0
